In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)

In [ ]:
fraud_df = pd.read_parquet('transaction_fraud_data.parquet')
fraud_df.head(5)

In [ ]:
# Чтобы посмотреть инфу о фрейме
fraud_df.info()

In [ ]:
# Некоторая описательная статистика по дата фрейму, которая поможет выявить выбросы и т.п. (аномальных значений не оказалось)
fraud_df.describe()

In [ ]:
# Проверка на Nan (на удивление нет пропущенных значений)
fraud_df.isna().sum()

In [ ]:
currency_df = pd.read_parquet('historical_currency_exchange.parquet')
currency_df.head(5)

In [ ]:
# Чтобы посмотреть инфу о фрейме
currency_df.info()

In [ ]:
# Некоторая описательная статистика по дата фрейму, которая поможет выявить выбросы и т.п. (аномальных значений не оказалось)
currency_df.describe()

In [ ]:
# Проверка на Nan (на удивление нет пропущенных значений)
currency_df.isna().sum()

Круговая диаграмма распределения НЕ fraud транзакций по типам вендоров

In [ ]:
# Подсчет количества транзакций по каждой категории вендоров
vendor_category_counts = fraud_df['vendor_category'][fraud_df.is_fraud==False].value_counts()

# Создание круговой диаграммы
def plot_pie_chart(df: pd.DataFrame, title: str):
    plt.figure(figsize=(10, 8))

    # Построение диаграммы с улучшенным стилем
    colors = plt.cm.Set3(range(len(df)))
    wedges, texts, autotexts = plt.pie(
        df.values, 
        labels=df.index,
        autopct='%1.1f%%',
        startangle=90,
        colors=colors,
        explode=[0.05] * len(df)  # Небольшой отступ для всех сегментов
    )

    # Настройка стиля текста
    for autotext in autotexts:
        autotext.set_color('white')
        autotext.set_fontweight('bold')

    plt.title(title, fontsize=16, pad=20)
    plt.axis('equal')  # Равные пропорции
    plt.show()

# Вывод числовых данных
def print_numeric_data(df: pd.DataFrame):
    print("Распределение по категориям вендоров:")
    print(df)
    print("\nПроцентное соотношение:")
    print((df / df.sum() * 100).round(2))


plot_pie_chart(vendor_category_counts, 'Распределение НЕ fraud транзакций по категориям вендоров')
print_numeric_data(vendor_category_counts)

Круговая диаграмма распределения fraud транзакций по типам вендоров

In [ ]:
# Подсчет количества транзакций по каждой категории вендоров
vendor_category_counts_fraud = fraud_df['vendor_category'][fraud_df.is_fraud==True].value_counts()

plot_pie_chart(vendor_category_counts_fraud, 'Распределение fraud транзакций по категориям вендоров')
print_numeric_data(vendor_category_counts_fraud)

Судя по круговым диаграммам, довольно симметричный датасет попался в процентном соотношениии

Гипотеза: Одно и то же device_fingerprint используется для множества транзакций разных клиентов

Ценность: Выявление подозрительных устройств и блокировка

В результате можно увидеть, что некоторые устройства используются разными людьми, что вызывает подозрения.

In [ ]:
# Считаем, сколько уникальных customer_id приходится на каждый device_fingerprint
device_risk = fraud_df.groupby('device_fingerprint').agg(
    num_customers=('customer_id', 'nunique'),
    num_transactions=('transaction_id', 'count'),
    fraud_count=('is_fraud', 'sum')
).reset_index()

# Добавим долю мошенничества
device_risk['fraud_rate'] = device_risk['fraud_count'] / device_risk['num_transactions']

# Отфильтруем устройства с более чем 1 клиентом
suspicious_devices = device_risk[device_risk['num_customers'] > 1]

# Посмотрим на топ-10 подозрительных устройств
suspicious_devices.sort_values(by='num_customers', ascending=False).head(10)

Гипотеза: Один и тот же IP используется для множества транзакций разных клиентов

Ценность: Создание черных списков IP или усиление проверок при повторном использовании

Получились довольно спорные результаты, по которым нельзя быть точно уверенным в том, чтобы занести определнные ip в черный список

In [ ]:
# Считаем, сколько уникальных customer_id приходится на каждый IP
ip_risk = fraud_df.groupby('ip_address').agg(
    num_customers=('customer_id', 'nunique'),
    num_transactions=('transaction_id', 'count'),
    fraud_count=('is_fraud', 'sum')
).reset_index()

# Добавим долю мошенничества
ip_risk['fraud_rate'] = ip_risk['fraud_count'] / ip_risk['num_transactions']

# Отфильтруем IP с более чем 1 клиентом
suspicious_ips = ip_risk[ip_risk['num_customers'] > 1]

# Посмотрим на топ-10 подозрительных IP
suspicious_ips.sort_values(by='num_customers', ascending=False).head(10)

Гипотеза: Несколько транзакций с одной и той же card_number могут указывать на мошенничество

Ценность: Предотвращение использования одной карты несколькими клиентами

В результате не было обнаружено карт, с которых совершали сделки хотя бы два униикальных клиента

In [ ]:
# Считаем, сколько уникальных customer_id приходится на каждый card_number
card_risk = fraud_df.groupby('card_number').agg(
    num_customers=('customer_id', 'nunique'),
    num_transactions=('transaction_id', 'count'),
    fraud_count=('is_fraud', 'sum')
).reset_index()

# Добавим долю мошенничества
card_risk['fraud_rate'] = card_risk['fraud_count'] / card_risk['num_transactions']

# Отфильтруем карты с более чем 1 клиентом
suspicious_cards = card_risk[card_risk['num_customers'] > 1]

# Посмотрим на топ-10 подозрительных карт
suspicious_cards.sort_values(by='num_customers', ascending=False).head(10)

Гипотеза: Транзакции вне страны клиента имеют значительно более высокий уровень мошенничества

Ценность: Возможность ввести усиленную аутентификацию для таких операций

В выводе даже без проведения корелляционных тестов видно, что большинство мошеннических операций совершается вне страны клинента

In [ ]:
# Группируем данные по признаку "транзакция вне страны клиента"
fraud_by_location = fraud_df.groupby('is_outside_home_country').agg(
    total_transactions=('transaction_id', 'count'),
    fraud_count=('is_fraud', 'sum')
).reset_index()

# Добавляем долю мошенничества
fraud_by_location['fraud_rate'] = fraud_by_location['fraud_count'] / fraud_by_location['total_transactions']

print(fraud_by_location)

In [ ]:
# Подготовим данные для графика
plot_data = fraud_by_location.copy()
plot_data['is_outside_home_country'] = plot_data['is_outside_home_country'].map({True: 'Да', False: 'Нет'})

# Доля мошенничества
plt.figure(figsize=(4, 3))
sns.barplot(data=plot_data, x='is_outside_home_country', y='fraud_rate', hue='is_outside_home_country')
plt.title('Доля мошеннических транзакций в зависимости от местоположения')
plt.xlabel('Транзакция вне страны клиента')
plt.ylabel('Доля мошенничества')
plt.ylim(0, 0.1)  # Ограничиваем для наглядности
plt.grid(axis='y')
plt.show()

Гипотеза: Премиальные карты (Gold Credit и т.п.) менее подвержены мошенничеству

Ценность: Сегментация клиентов по уровню риска и предложение персонализированных продуктов

In [ ]:
# Группируем данные по типу карты
fraud_by_card_type = fraud_df.groupby('card_type').agg(
    total_transactions=('transaction_id', 'count'),
    fraud_count=('is_fraud', 'sum')
).reset_index()

# Добавляем долю мошенничества
fraud_by_card_type['fraud_rate'] = fraud_by_card_type['fraud_count'] / fraud_by_card_type['total_transactions']

# Сортируем по убыванию уровня мошенничества для наглядности
fraud_by_card_type = fraud_by_card_type.sort_values('fraud_rate', ascending=False)

print(fraud_by_card_type)

Как видно из вывода выше, доля мошеннических операций распределена по типам карт одинаково. Можно дополнительно проверить с помощью статистики.

In [ ]:
from scipy.stats import chi2_contingency

# Создаем таблицу сопряженности
contingency_table = pd.crosstab(fraud_df['card_type'], fraud_df['is_fraud'])

# Хи-квадрат тест
alpha = 0.05
chi2, p_value, dof, expected = chi2_contingency(contingency_table)

if p_value > alpha:
    print(f"Корреляция card_type и is_fraud == 0")
else:
    print(f"Корреляция card_type и is_fraud != 0")

“Страна клиента vs. страна транзакции” — heatmap подозрительных перемещений, который помогает выявить пары стран, где часто происходит fraud. В результате можно усилить контроль за операциями в странах из которых происходит fraud

In [ ]:
# Оставляем только мошеннические транзакции
fraud_only = fraud_df[fraud_df["is_fraud"] == True].copy()

# Определяем "домашнюю страну" клиента как наиболее частую страну его транзакций
home_country = (
    fraud_df.groupby("customer_id")["country"]
            .agg(lambda x: x.value_counts().index[0])
            .reset_index()
            .rename(columns={"country": "home_country"})
)

# Добавляем домашнюю страну в fraud-транзакции
fraud_only = fraud_only.merge(home_country, on="customer_id", how="left")

# Группировка: домашняя страна vs. фактическая страна транзакции
pivot = (
    fraud_only.groupby(["home_country", "country"])
              .size()
              .reset_index(name="fraud_count")
)

# Превращаем в матрицу для heatmap
matrix = pivot.pivot(index="home_country", columns="country", values="fraud_count").fillna(0)

# Визуализация
plt.figure(figsize=(14,8))
sns.heatmap(matrix, cmap="Reds", linewidths=0.5)
plt.title("Heatmap: Home Country vs. Transaction Country (FRAUD only)", fontsize=14)
plt.xlabel("Страна транзакции")
plt.ylabel("Домашняя страна клиента")
plt.show()